In [1]:
import sqlite3

import semver


def queryall(con, sql):
    return list(con.execute(sql))


con = sqlite3.connect("jan.db")

print(list(con.execute("select sqlite_version()")))

[('3.37.2',)]


In [5]:
# print(queryall(con, 'select count(*) from file_hash'))

# queryall(con, "SELECT load_extension('/usr/lib/sqlite3/pcre.so');")

sql_all_releases = """
select distinct(release) from file_hash
where project_name='flask'
"""
ALL_RELS = [row[0] for row in queryall(con, sql_all_releases)]
print(ALL_RELS)

['0.1', '0.10', '0.11', '0.12', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9', '1.0', '1.1.0', '2.0.0']


In [22]:
RELS = ["0.2", "0.6", "1.0", "2.0.0"]
RELS = [
    "0.1",
    "0.2",
    "0.3",
    "0.4",
    "0.5",
    "0.6",
    "0.7",
    "0.8",
    "0.9",
    "0.10",
    "0.11",
    "0.12",
    "1.0",
]
# major change -> '1.1.0'
# '1.0', '1.1.0', '2.0.0'
# insert into file_hash(path, hash, size_bytes, release, project_name)

sql = """
select path, size_bytes from file_hash
where release='{0}'
and project_name='flask'
and path like '%.py'
and path not like 'examples/%'
and path not like 'tests/%'
and path not like 'docs/%'
and path not like '%testsuite/%'
"""

# rels = ALL_RELS
rels = RELS
# rels = ['0.5', '0.6']

release = {}
for release_ver in rels:
    data = queryall(con, sql.format(release_ver))
    data = dict((name, int(size)) for name, size in data)
    release[release_ver] = data
    num_files = len(data)
    # print(f'RELEASE: {release_ver:7} - {num_files} files')

# assert 0, set(release['1.0']) ^ set(release['1.1.0'])

last = release[rels[-1]]
last_files = list(sorted(last.keys()))

grid = {}
for release_ver in rels:
    this_rel = release[release_ver]
    print(release_ver, this_rel)
    this_slice = [this_rel.get(path, "-") for path in last_files]
    # assert 0, this_slice
    grid[release_ver] = this_slice

for release_ver in rels:
    print(f"{release_ver:6} {grid[release_ver]}")

0.1 {'flask.py': 25871, 'setup.py': 1571}
0.2 {'flask.py': 43971, 'setup.py': 1591}
0.3 {'flask.py': 55220, 'setup.py': 1591}
0.4 {'flask.py': 59156, 'setup.py': 1789}
0.5 {'flask/__init__.py': 918, 'flask/app.py': 33506, 'flask/config.py': 5542, 'flask/ctx.py': 2098, 'flask/globals.py': 579, 'flask/helpers.py': 14669, 'flask/logging.py': 1313, 'flask/module.py': 7544, 'flask/session.py': 1268, 'flask/templating.py': 2710, 'flask/testing.py': 1356, 'flask/wrappers.py': 2353, 'setup.py': 1787}
0.6 {'flask/__init__.py': 1100, 'flask/app.py': 36667, 'flask/config.py': 5542, 'flask/ctx.py': 2095, 'flask/globals.py': 579, 'flask/helpers.py': 17136, 'flask/logging.py': 1312, 'flask/module.py': 8363, 'flask/session.py': 1269, 'flask/signals.py': 1826, 'flask/templating.py': 3038, 'flask/testing.py': 1356, 'flask/wrappers.py': 3026, 'setup.py': 1787}
0.7 {'flask/__init__.py': 1225, 'flask/app.py': 55236, 'flask/blueprints.py': 12791, 'flask/config.py': 5980, 'flask/ctx.py': 5540, 'flask/global